In [58]:
%pip install pythautomata==0.21.1


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Creating a moore machine

#### Dependencies

In [59]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.moore_state import MooreState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy
from pythautomata.model_exporters.dot_exporting_mm_strategy import DotExportingMMStrategy

#### Implementation

In [60]:
# Input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

# Define states
stateA = MooreState("State A", output_alphabet['0'])
stateB = MooreState("State B", output_alphabet['1'])
stateC = MooreState("State C", output_alphabet['1'])

# Add transitions
stateA.add_transition(a, stateA)
stateA.add_transition(b, stateB)
stateB.add_transition(a, stateC)
stateB.add_transition(c, stateB)

# Hole state
hole_state = MooreState(name="hole", value=SymbolStr('0'))

moore_machine = MooreMachineAutomaton(input_alphabet, output_alphabet, stateA, set([stateA, stateB, stateC]), MooreMachineComparisonStrategy, "moore machine with 3 states", [DotExportingMMStrategy])

# Moore machine is generated in the directory ./output_models/moore_machines/
moore_machine.export('./output_models/moore_machines/')

Unexpected exception when exporting moore machine with 3 states: <class 'AttributeError'>


### Moore machine generator

#### Dependencies

In [61]:
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine

#### Implementation

In [62]:
# Amount of states
size = 20

generated_moore_machine = generate_moore_machine(input_alphabet, output_alphabet, size)

# Moore machine is generated in the directory ./output_models/moore_machines/
generated_moore_machine.export('./output_models/moore_machines/')

### DFA - moore machine comparator and converter

#### Dependencies

In [63]:
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines

#### Implementation

In [64]:
# Tomitas DFA example
tomitas = TomitasGrammars.get_automaton_1()

# Convert DFA to moore machine
converted_moore_machine = AutomataConverter.convert_dfa_to_moore_machine(tomitas)

# Tomitas moore machine
tomitas_moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Comparate automatons
converted_moore_machine.__eq__(tomitas_moore_machine)

True

In [65]:
%pip install pymodelextractor==0.16.0


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Moore machine L*

#### Dependencies

In [66]:
import time
from pymodelextractor.learners.observation_table_learners.mm_lstar_learner import MMLStarLearner as MooreMachineLearner
from pymodelextractor.learners.observation_table_learners.lstar_learner import LStarLearner as DFALearner
from pymodelextractor.teachers.moore_machines_teacher import MooreMachineTeacher as MMTeacher
from pymodelextractor.teachers.automaton_teacher import DeterministicFiniteAutomatonTeacher as DFATeacher
from pythautomata.model_comparators.dfa_comparison_strategy import DFAComparisonStrategy
from pythautomata.automata.deterministic_finite_automaton import DeterministicFiniteAutomaton
from pythautomata.utilities.nicaud_dfa_generator import generate_dfa

#### Implementation

In [67]:
def get_mm_learner():
    return MooreMachineLearner()

def get_mm_teacher(automaton: MooreMachineAutomaton) -> MMTeacher:
    return MMTeacher(automaton, MooreMachineComparisonStrategy())

def get_dfa_learner():
    return DFALearner()

def get_dfa_teacher(automaton: DeterministicFiniteAutomaton) -> DFATeacher:
    return DFATeacher(automaton, DFAComparisonStrategy())

In [68]:
def test_500_states_moore_machine():
    strings = ["0", "1"]
    input_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    strings = ["a", "b", "c"]
    output_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    seed = 100
    automata = generate_moore_machine(input_alphabet, output_alphabet, 500, seed)
    teacher = get_mm_teacher(automata)
    start_time = time.time()
    result = get_mm_learner().learn(teacher)
    duration = time.time() - start_time
    print("MM Learner Duration for 500 states: ", duration, "s")
    assert MooreMachineComparisonStrategy().are_equivalent(
        result.model, automata)

In [69]:
def test_500_states_dfa():
    strings = ["0", "1"]
    input_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    seed = 100
    automata = generate_dfa(input_alphabet, 500, seed)
    teacher = get_dfa_teacher(automata)
    start_time = time.time()
    result = get_dfa_learner().learn(teacher)
    duration = time.time() - start_time
    print("DFA Learner Duration for 500 states: ", duration, "s")
    assert DFAComparisonStrategy().are_equivalent(
        result.model, automata)

In [70]:
def test_1000_states_moore_machine():
    strings = ["0", "1"]
    input_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    strings = ["a", "b", "c"]
    output_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    seed = 100
    automata = generate_moore_machine(input_alphabet, output_alphabet, 1000, seed)
    teacher = get_mm_teacher(automata)
    start_time = time.time()
    result = get_mm_learner().learn(teacher)
    duration = time.time() - start_time
    print("MM Learner Duration for 1000 states: ", duration, "s")
    assert MooreMachineComparisonStrategy().are_equivalent(
        result.model, automata)

In [71]:
def test_1000_states_dfa():
    strings = ["0", "1"]
    input_alphabet = Alphabet(frozenset(map(SymbolStr, strings)))
    seed = 100
    automata = generate_dfa(input_alphabet, 1000, seed)
    teacher = get_dfa_teacher(automata)
    start_time = time.time()
    result = get_dfa_learner().learn(teacher)
    duration = time.time() - start_time
    print("DFA Learner Duration for 1000 states: ", duration, "s")
    assert DFAComparisonStrategy().are_equivalent(
        result.model, automata)

In [72]:
test_500_states_moore_machine()
test_500_states_dfa()
test_1000_states_moore_machine()
test_1000_states_dfa()

MM Learner Duration for 500 states:  24.04143738746643 s
DFA Learner Duration for 500 states:  119.05761003494263 s
MM Learner Duration for 1000 states:  109.61600613594055 s
